<a href="https://colab.research.google.com/github/annvorosh/GB/blob/DAmed/DAmed_L05_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Введение в анализ данных в медицине
## Урок 5. Персонифицированная и предиктивная медицина

В онлайн-инструменте PyHealth выберите наиболее интересующий вас пайплайн (например, Mortality Prediction) и попробуйте его прогнать (графа Tutorials) на предложенных разработчиками датасетах (графа Datasets) стартовой страницы.

Выберите модель, подходящую для вашей задачи, аргументируйте, почему именно эта модель кажется вам наилучшей. Обучите модель и протестируйте ее, оцените качество.

Мы выбираем задачу прогнозирования смертности пациента и пользуемся доступным датасетом, например, MIMIC-III.
https://pyhealth.readthedocs.io/en/latest/index.html

In [2]:
!pip install pyhealth

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.3/226.3 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.5/30.5 MB 24.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 82.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.9/140.9 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.9 MB/s eta 0:00:00
  Created wheel for pandarallel: filename=pandarallel-1.6.5-py3-none-any.whl size=16672 sha256=ca4d664452a4033d5dbc4877a669f96aefde389d79b028f886d5e16ce9a16dc6
  Stored in directory: /root/.cache/pip/wheels/50/4f/1e/34e057bb868842209f1623f195b74fd7eda229308a7352d47f
Successfully built pandarallel
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.0.7
    Uninstalling urllib3-2.0.7:
      Successfully uninstalled urllib3-2.0.7


In [3]:
from pyhealth.datasets import MIMIC3Dataset
mimic3base = MIMIC3Dataset(
    root="https://storage.googleapis.com/pyhealth/Synthetic_MIMIC-III/",
    tables=["DIAGNOSES_ICD", "PROCEDURES_ICD", "PRESCRIPTIONS"],
    # map all NDC codes to ATC 3-rd level codes in these tables
    code_mapping={"NDC": ("ATC", {"target_kwargs": {"level": 3}})},
)



INFO: Pandarallel will run on 1 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
finish basic patient information parsing : 90.90874409675598s
finish parsing DIAGNOSES_ICD : 63.937361001968384s
finish parsing PROCEDURES_ICD : 51.48145365715027s
finish parsing PRESCRIPTIONS : 163.49584364891052s


Mapping codes: 100%|██████████| 49993/49993 [00:55<00:00, 902.69it/s] 


###  Выберем задачу предсказания рекомендации по лекарствам на основе информации из MIMIC-III.

In [33]:
from pyhealth.tasks import drug_recommendation_mimic3_fn
from pyhealth.datasets import split_by_patient, get_dataloader

mimic3sample = mimic3base.set_task(task_fn=drug_recommendation_mimic3_fn) # use default task
train_ds, val_ds, test_ds = split_by_patient(mimic3sample, [0.8, 0.1, 0.1])

# create dataloaders (torch.data.DataLoader)
train_loader = get_dataloader(train_ds, batch_size=32, shuffle=True)
val_loader = get_dataloader(val_ds, batch_size=32, shuffle=False)
test_loader = get_dataloader(test_ds, batch_size=32, shuffle=False)

Generating samples for drug_recommendation_mimic3_fn: 100%|██████████| 49993/49993 [00:03<00:00, 14687.12it/s]


In [34]:
from pyhealth.models import Transformer

model = Transformer(
    dataset=mimic3sample,
    feature_keys=["conditions", "procedures"],
    label_key="drugs",
    mode="multilabel",
)

In [35]:
from pyhealth.trainer import Trainer

trainer = Trainer(model=model)
trainer.train(
    train_dataloader=train_loader,
    val_dataloader=val_loader,
    epochs=50,
    monitor="pr_auc_samples",
)

Transformer(
  (embeddings): ModuleDict(
    (conditions): Embedding(2148, 128, padding_idx=0)
    (procedures): Embedding(661, 128, padding_idx=0)
  )
  (linear_layers): ModuleDict()
  (transformer): ModuleDict(
    (conditions): TransformerLayer(
      (transformer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadedAttention(
            (linear_layers): ModuleList(
              (0-2): 3 x Linear(in_features=128, out_features=128, bias=False)
            )
            (output_linear): Linear(in_features=128, out_features=128, bias=False)
            (attention): Attention()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (feed_forward): PositionwiseFeedForward(
            (w_1): Linear(in_features=128, out_features=512, bias=True)
            (w_2): Linear(in_features=512, out_features=128, bias=True)
            (dropout): Dropout(p=0.5, inplace=False)
            (activation): GELU(approximate='none')
          )
          (i

/usr/local/lib/python3.10/dist-packages/pyhealth/trainer.py:12: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import trange
INFO:pyhealth.trainer:Transformer(
  (embeddings): ModuleDict(
    (conditions): Embedding(2148, 128, padding_idx=0)
    (procedures): Embedding(661, 128, padding_idx=0)
  )
  (linear_layers): ModuleDict()
  (transformer): ModuleDict(
    (conditions): TransformerLayer(
      (transformer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadedAttention(
            (linear_layers): ModuleList(
              (0-2): 3 x Linear(in_features=128, out_features=128, bias=False)
            )
            (output_linear): Linear(in_features=128, out_features=128, bias=False)
            (attention): Attention()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (feed_forward): PositionwiseFeedForward(
 

Metrics: None


INFO:pyhealth.trainer:Metrics: None


Device: cpu


...

INFO:pyhealth.trainer:pr_auc_samples: 0.5422


loss: 0.2721


INFO:pyhealth.trainer:loss: 0.2721


Loaded best model


INFO:pyhealth.trainer:Loaded best model


In [36]:
trainer.evaluate(test_loader)

Evaluation: 100%|██████████| 6/6 [00:00<00:00, 79.72it/s]


{'pr_auc_samples': 0.557524764783103, 'loss': 0.3237651437520981}

In [41]:
from pyhealth.models import CNN, RNN, Transformer, RETAIN

# Создание и обучение моделей
models = [CNN, RNN, Transformer, RETAIN]
results = {}

for model_class in models:
    model = model_class(
        dataset=mimic3sample,
        feature_keys=["conditions", "procedures"],
        label_key="drugs",
        mode="multilabel",
    )

    trainer = Trainer(model=model)
    trainer.train(
        train_dataloader=get_dataloader(train_ds, batch_size=32, shuffle=True),
        val_dataloader=get_dataloader(val_ds, batch_size=32, shuffle=False),
        epochs=50,
        monitor="pr_auc_samples",
    )

    evaluation_result = trainer.evaluate(get_dataloader(test_ds, batch_size=32, shuffle=False))
    results[model_class.__name__] = evaluation_result

CNN(
  (embeddings): ModuleDict(
    (conditions): Embedding(2148, 128, padding_idx=0)
    (procedures): Embedding(661, 128, padding_idx=0)
  )
  (linear_layers): ModuleDict()
  (cnn): ModuleDict(
    (conditions): CNNLayer(
      (cnn): ModuleDict(
        (CNN-0): CNNBlock(
          (conv1): Sequential(
            (0): Conv1d(128, 128, kernel_size=(3,), stride=(1,), padding=(1,))
            (1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): ReLU()
          )
          (conv2): Sequential(
            (0): Conv1d(128, 128, kernel_size=(3,), stride=(1,), padding=(1,))
            (1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          )
          (relu): ReLU()
        )
      )
      (pooling): AdaptiveAvgPool1d(output_size=1)
    )
    (procedures): CNNLayer(
      (cnn): ModuleDict(
        (CNN-0): CNNBlock(
          (conv1): Sequential(
            (0): Conv1d(128, 128, kernel_size=(3,

INFO:pyhealth.trainer:CNN(
  (embeddings): ModuleDict(
    (conditions): Embedding(2148, 128, padding_idx=0)
    (procedures): Embedding(661, 128, padding_idx=0)
  )
  (linear_layers): ModuleDict()
  (cnn): ModuleDict(
    (conditions): CNNLayer(
      (cnn): ModuleDict(
        (CNN-0): CNNBlock(
          (conv1): Sequential(
            (0): Conv1d(128, 128, kernel_size=(3,), stride=(1,), padding=(1,))
            (1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): ReLU()
          )
          (conv2): Sequential(
            (0): Conv1d(128, 128, kernel_size=(3,), stride=(1,), padding=(1,))
            (1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          )
          (relu): ReLU()
        )
      )
      (pooling): AdaptiveAvgPool1d(output_size=1)
    )
    (procedures): CNNLayer(
      (cnn): ModuleDict(
        (CNN-0): CNNBlock(
          (conv1): Sequential(
            (0): Conv1d(128

Metrics: None


INFO:pyhealth.trainer:Metrics: None


Device: cpu


...

INFO:pyhealth.trainer:pr_auc_samples: 0.5356


loss: 0.2615


INFO:pyhealth.trainer:loss: 0.2615


Loaded best model


INFO:pyhealth.trainer:Loaded best model
Evaluation: 100%|██████████| 6/6 [00:00<00:00, 62.84it/s]


In [45]:
# Вывод результатов
for model_name, result in results.items():
    print(f"\n\nModel: {model_name}")
    print(result)



Model: CNN
{'pr_auc_samples': 0.5652021137228372, 'loss': 0.2855441942811012}


Model: RNN
{'pr_auc_samples': 0.5818098499791743, 'loss': 0.24511979520320892}


Model: Transformer
{'pr_auc_samples': 0.5545592924842471, 'loss': 0.31671397884686786}


Model: RETAIN
{'pr_auc_samples': 0.5740790155950405, 'loss': 0.2752765615781148}


Bсе модели показали неплохие результаты, но RNN показала самое низкое значение потерь и немного более высокий показатель PR AUC по сравнению с другими моделями. Это может говорить о том, что RNN лучше всего справилась с задачей на наших данных.

RNN, или рекуррентные нейронные сети, часто применяются в анализе временных последовательностей, что может быть полезно в задачах, связанных с медицинскими данными. Эти сети хорошо работают с последовательными данными, такими как временные ряды, тексты или сигналы, благодаря способности запоминать информацию о предыдущих состояниях.

Когда речь идет о медицинских данных, которые обычно имеют временной компонент и сильно зависят от предыдущих состояний пациента (например, последовательность событий в истории болезни), RNN могут успешно выявлять и учитывать эти зависимости.